## 多品種輸送問題

In [1]:
import numpy as np
import pandas as pd
from mypulp import *
import csv

In [2]:
# 工場で製造可能な製品を抽出
df_p = pd.read_csv('constraints/Multi-product_transportation_problem/produce.csv')
# DataFrameをdictへ変換
produce = df_p.set_index('factory').T.to_dict('list') 
# Nanを削除
for key in produce.keys():
    produce[key] = {x for x in produce[key] if x==x}

In [3]:
# 顧客と商品の需要量を抽出
df_d = pd.read_csv('constraints/Multi-product_transportation_problem/demand.csv')
# 顧客番号と製品番号のタプル（組）を作成
D = list(zip(df_d[df_d.columns[0]], df_d[df_d.columns[1]]))
# 顧客番号と製品番号のタプル（組）をキーとし、需要量を値とする辞書を作成
d = dict(zip(D, df_d[df_d.columns[2]]))

In [4]:
# 顧客番号の集合Iを生成
I = set([i for (i,k) in d])
# 工場の番号リストJと生産容量Mをmultidictを用いて作成
J, M = multidict({1:3000, 2:3000, 3:3000})
# 製品番号リストKと重量weightをmultidictを用いて作成
K, weight = multidict({1:5, 2:2, 3:3, 4:4})

In [5]:
# 顧客と商品の輸送費用を抽出
df_c = pd.read_csv('constraints/Multi-product_transportation_problem/cost.csv')
# 顧客番号と製品番号のタプル（組）を作成
C = list(zip(df_c[df_c.columns[0]], df_c[df_c.columns[1]]))
# 顧客番号と製品番号のタプル（組）をキーとし、輸送費用を値とする辞書を作成
cost = dict(zip(C, df_c[df_c.columns[2]]))

In [6]:
# weightとcostから製品毎の輸送費用を計算し、辞書cに保管
c = {}
for i in I:
    for j in J:
        for k in produce[j]:
            c[i, j, k] = cost[i, j] * weight[k]

In [7]:
# 各変数の確認
# produce = {1:[2,4], 2:[1,2,3], 3:[2,3,4]}
# d = {(1,1):80,   (1,2):85,   (1,3):300, (1,4):6,
#      (2,1):270,  (2,2):160,  (2,3):400, (2,4):7,
#      (3,1):250,  (3,2):130,  (3,3):350, (3,4):4,
#      (4,1):160,  (4,2):60,   (4,3):200, (4,4):3,
#      (5,1):180,  (5,2):40,   (5,3):150, (5,4):5
#     }
# I = {1, 2, 3, 4, 5}
# J = [1, 2, 3]
# M = {1:3000, 2:3000, 3:3000}
# K = [1, 2, 3, 4]
# weight = {1:5, 2:2, 3:3, 4:4}
# cost = {(1,1):4,  (1,2):6,  (1,3):9,
#         (2,1):5,  (2,2):4,  (2,3):7,
#         (3,1):6,  (3,2):3,  (3,3):4,
#         (4,1):8,  (4,2):5,  (4,3):3,
#         (5,1):10, (5,2):8,  (5,3):4,
#        }

In [8]:
# モデルの定義
model = Model(name = "Multi-product_transportation_problem")
# 変数を格納する辞書xを作成
# 変数は輸送費用を表す辞書cのキーが存在する場合にだけ生成
x = {}
for i,j,k in c:
    x[i,j,k] = model.addVar(vtype="C")
model.update()

In [9]:
# 需要満足条件
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i,j,k] for j in J if (i,j,k) in x)==d[i,k])

# 工場の容量制約
for j in J:
    model.addConstr(quicksum(x[i,j,k] for (i,j2,k) in x if j2 == j) <= M[j])

In [10]:
# 目的関数
model.setObjective(quicksum(c[i,j,k]*x[i,j,k] for (i,j,k) in x), GRB.MINIMIZE)
# 最適化の実行
model.optimize()
print("Opt value:", model.ObjVal)

Opt value: 43536.0
